# ECS 171 Group Project - Group 10 - ???

## Clean/Wrangle Data

"Process the data the same way the paper author did"

In [6]:
#dont run this unless you are hosting data anad notebook in google drive
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/My Drive/171-g1/notebook') #change this path with folder containing your code

Mounted at /content/drive/


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
import pandas as pd


pd.options.display.max_columns = 999

#Contains 81 features extracted from 21263 superconductors along with the critical temperature in the 82nd column
train = pd.read_csv('./../superconduct/train.csv')

#Contains the chemical formula broken up for all the 21263 superconductors from the train.csv file.
#The last two columns have the critical temperature and chemical formula.
formula = pd.read_csv('./../superconduct/unique_m.csv')

print("Training Data")
print(train.shape)
print(train.head())

print("\nChemical formulas")
print(formula.shape)
print(formula.head())

## Reference Neural Network

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

trainx = train.drop("critical_temp",axis=1)
trainy = train['critical_temp']

train_X, test_X, train_y, test_y = train_test_split(trainx,trainy, train_size=0.7, random_state=1)

sc = MinMaxScaler()

train_X = sc.fit_transform(train_X)
test_X = sc.fit_transform(test_X)

In [ ]:
nn = Sequential(
    [
        keras.Input(shape=train_X.shape[1]),
        layers.Dense(20, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(20, activation='relu'),
        layers.Dense(1)
    ]
)

nn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
           loss="mean_squared_error",
           metrics=[keras.metrics.MeanSquaredError()])

history = nn.fit(train_X, train_y, batch_size=64, epochs=700, verbose=0)

pred = nn.predict(test_X)

print(history.history['mean_squared_error'])

In [ ]:
import matplotlib.pyplot as plt

# showing graph with first 100 values
plt.figure(figsize=(50,50))
line1 = plt.plot(range(100), pred[:100],linewidth=1, label='prediction')
line2 = plt.plot(range(100), train_y[:100], label='actual')
plt.legend(loc="upper left", prop={'size': 60})

In [ ]:
# source https://www.pyimagesearch.com/2019/01/21/regression-with-keras/
diff = pred.flatten() - train_y
percent_diff = (diff/train_y)*100
abs_percent_diff = np.abs(percent_diff)
mean = np.mean(abs_percent_diff)
std = np.std(abs_percent_diff)
print("Mean Abs. Percent Diff: " + str(mean))
print("Std. Deviation        : " + str(std))

## Replicate original XGBoost model

## Optimize XGBoost model with tuned hyperparameters

## Train and test a Random Forest model

## Optimize Random Forest model with tuned hyperparameters

## Research Bayesian Neural Networks
## Train and test a Bayesian Neural Network

In [ ]:
!pip install blitz-bayesian-pytorch

In [9]:
# Code  source for BNN https://towardsdatascience.com/making-your-neural-network-say-i-dont-know-bayesian-nns-using-pyro-and-pytorch-b1c24e6ab8cd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator

from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from blitz.modules.base_bayesian_module import BayesianModule

from blitz.modules.weight_sampler import TrainableRandomDistribution
from blitz.losses import kl_divergence_from_nn
from blitz.modules.base_bayesian_module import BayesianModule, BayesianRNN
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [12]:
X = train.drop("critical_temp", axis = 1)
y = train['critical_temp']
X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(np.expand_dims(y, -1))

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = .3,
                                                    random_state = 42)


X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).float()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test).float()

In [13]:
X_train.shape

torch.Size([14884, 81])

In [14]:
#variational_estimator
class BayesianRegressor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        #self.linear = nn.Linear(input_dim, output_dim)
        self.blinear1 = BayesianLinear(input_dim, 100)
        self.blinear2 = BayesianLinear(100, output_dim)
        
    def forward(self, x):
        x_ = self.blinear1(x)
        return self.blinear2(x_)
    
    
#from https://github.com/piEsposito/blitz-bayesian-deep-learning/blob/3526b57058e0914105e17cf7e93682ef11a9cd28/blitz/utils/variational_estimator.py#L35    
    def nn_kl_divergence(self):
        """Returns the sum of the KL divergence of each of the BayesianModules of the model, which are from
            their posterior current distribution of weights relative to a scale-mixtured prior (and simpler) distribution of weights
            Parameters:
                N/a
            Returns torch.tensor with 0 dim.      
        
        """
        return kl_divergence_from_nn(self)
    
    def sample_elbo(self,
                    inputs,
                    labels,
                    criterion,
                    sample_nbr,
                    complexity_cost_weight=1):

        """ Samples the ELBO Loss for a batch of data, consisting of inputs and corresponding-by-index labels
                The ELBO Loss consists of the sum of the KL Divergence of the model 
                 (explained above, interpreted as a "complexity part" of the loss)
                 with the actual criterion - (loss function) of optimization of our model
                 (the performance part of the loss). 
                As we are using variational inference, it takes several (quantified by the parameter sample_nbr) Monte-Carlo
                 samples of the weights in order to gather a better approximation for the loss.
            Parameters:
                inputs: torch.tensor -> the input data to the model
                labels: torch.tensor -> label data for the performance-part of the loss calculation
                        The shape of the labels must match the label-parameter shape of the criterion (one hot encoded or as index, if needed)
                criterion: torch.nn.Module, custom criterion (loss) function, torch.nn.functional function -> criterion to gather
                            the performance cost for the model
                sample_nbr: int -> The number of times of the weight-sampling and predictions done in our Monte-Carlo approach to 
                            gather the loss to be .backwarded in the optimization of the model.        
        
        """

        loss = 0
        for _ in range(sample_nbr):
            outputs = self(inputs)
            loss += criterion(outputs, labels) 
            loss += self.nn_kl_divergence() * complexity_cost_weight
            
        return loss / sample_nbr

In [22]:
def evaluate_regression(regressor,
                        Xt,
                        yt,
                        samples = 100,
                        std_multiplier = 2):
    preds = [regressor(Xt) for i in range(samples)]
    preds = torch.stack(preds)
    means = preds.mean(axis=0)
    stds = preds.std(axis=0)
    ci_upper = means + (std_multiplier * stds)
    ci_lower = means - (std_multiplier * stds)
    ic_acc = (ci_lower <= yt) * (ci_upper >= yt)
    ic_acc = ic_acc.float().mean()
    return ic_acc, (ci_upper >= yt).float().mean(), (ci_lower <= yt).float().mean()

In [23]:
regressor = BayesianRegressor(81, 1)
optimizer = optim.Adam(regressor.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()


ds_train = torch.utils.data.TensorDataset(X_train, y_train)
dataloader_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)

ds_test = torch.utils.data.TensorDataset(X_test, y_test)
dataloader_test = torch.utils.data.DataLoader(ds_test, batch_size=32, shuffle=True)

In [24]:

iteration = 0
for epoch in range(100):
    for i, (datapoints, labels) in enumerate(dataloader_train):
        optimizer.zero_grad()
        
        loss = regressor.sample_elbo(inputs=datapoints,
                           labels=labels,
                           criterion=criterion,
                           sample_nbr=3)

        loss.backward()
        optimizer.step()
        
        iteration += 1
        if iteration%100==0:
            ic_acc, under_ci_upper, over_ci_lower = evaluate_regression(regressor,
                                                                        X_test,
                                                                        y_test,
                                                                        samples=32,
                                                                        std_multiplier=3)
            
            print("CI acc: {:.2f}, CI upper acc: {:.2f}, CI lower acc: {:.2f}".format(ic_acc, under_ci_upper, over_ci_lower))
            print("Loss: {:.4f}".format(loss))

CI acc: 0.01, CI upper acc: 0.64, CI lower acc: 0.37
Loss: 35274.5938
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 34306.2891
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 33853.0977
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 33607.4883
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 33283.9531
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 32897.2617
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 32610.0469
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 32258.3203
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 31967.0605
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 31691.2246
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 31328.4785
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 31050.8691
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 30694.2637
CI acc: 0.00, CI upper acc: 0.63, CI lower acc: 0.37
Loss: 30331.5098
CI acc: 0.00, CI upp

## Optimize Bayesian Neural Network with tuned hyperparameters

## Analyze all model results and performances

## Frontend?